In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os, sys
PATH = os.path.abspath("")[:-3]

In [2]:

# SCRAPPEAR UN USUARIO
from test_user import get_user_data

USER = "ComicGirlAshley"
user_data = get_user_data(USER)
print(user_data)

[340, False, 26626, 97, 19968, 134, 6255, 4.637254901960785, 17.470588235294116, 271.45098039215685, 0.8921568627450981, 0.1568627450980392, 26529]


c:\Users\maria\Documents\Uni\ia orgs\bot_detector\.venv\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [3]:
# Se leen los datos generados con model.py\n",
df_data = pd.read_csv(PATH + "data/training_new.csv").sample(frac=1).reset_index(drop=True)
data = df_data.iloc[:, 1:]
print(data.head())

   factor_rep  verified  followersCount  friendsCount  tweetsCount  \
0         822         0             387           401         1349   
1         256         1          279018          6918        90603   
2         628         1          734610           435        22071   
3         324         0             184           332         1157   
4         196         1          131878          2269          260   

   listedCount  mediaCount  avg_reply  avg_retweet   like_reply  avg_quote  \
0            0         825   0.081081     0.729730     2.648649   0.054054   
1         1117        6146  49.254902   343.058824  1113.313725  17.156863   
2         9121        7977   1.352941    18.372549    38.941176   2.549020   
3            3           5   0.615385     0.192308     5.076923   0.115385   
4         3402        1785   1.365385     0.192308     3.634615   0.019231   

   hora_rep  followers_diff  clase  
0  0.243243            -387    bot  
1  0.156863         -279017  human  

In [4]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
header_names = list(data.columns.values)
for col in data.iloc[:,:-1]:
    data[col] = preprocessing.MinMaxScaler().fit_transform(np.array(data[col]).reshape(-1,1))
x_data = data.iloc[:,:-1].to_numpy()
y_data = data.iloc[:,-1].to_numpy()
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, stratify=y_data, test_size=0.33)

In [5]:
#CONVERTIRLA SALIDA A UN VECTOR DE 0 Y 1
from sklearn.preprocessing import LabelBinarizer

encoder=LabelBinarizer()
label = encoder.fit_transform(y_train)
y_train_transformed = np.hstack((label, 1 - label))
label = encoder.fit_transform(y_test)
y_test_transformed = np.hstack((label, 1 - label))
print(y_train[:5])
print(y_train_transformed[:5])

['human' 'bot' 'human' 'bot' 'bot']
[[1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]]


In [6]:
# COMPROBAR TIPO DE LOS DATOS
print(type(x_train))
print(type(y_train_transformed))
# COMPROBAR DIMENSIONES DE LOS DATOS
print(x_train.shape)
print(y_train.shape)
print(y_train_transformed.shape)
print(x_test.shape)
print(y_test.shape)
print(y_test_transformed.shape) 
# OBTENER DIMENSION DE LA ENTRADA Y NÚMERO DE SALIDAS
input_shape = (x_train.shape[1] ,)
num_clases = y_test_transformed.shape[1]

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(1616, 13)
(1616,)
(1616, 2)
(796, 13)
(796,)
(796, 2)


# Buscar modelo

In [76]:
# Crear modelo usando sigmoide
from keras.models import Sequential
from keras.layers import Dense
input_shape= (x_train.shape[1] ,)
model = Sequential()
arch = [50, 100, 0, "relu", 0.2]
model.add(Dense(arch[0], input_shape=input_shape, activation=arch[3]))
if arch[1] != 0:
    model.add(Dense(arch[1], input_shape=input_shape, activation=arch[3]))
if arch[2] != 0:
    model.add(Dense(arch[2], input_shape=input_shape, activation=arch[3]))
model.add(Dense(2, activation='softmax'))
model.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_72 (Dense)            (None, 50)                700       
                                                                 
 dense_73 (Dense)            (None, 100)               5100      
                                                                 
 dense_74 (Dense)            (None, 2)                 202       
                                                                 
Total params: 6,002
Trainable params: 6,002
Non-trainable params: 0
_________________________________________________________________


In [77]:
# CONFIGURAR MODELO Y ENTRENAMIENTO
from keras.callbacks import EarlyStopping

epochs = 3000
early_stopping = EarlyStopping(monitor='val_loss', patience=1)
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=arch[4], momentum=0), metrics=['accuracy','mse'], loss='mean_squared_error')
historico = model.fit(x_train, y_train_transformed, epochs=epochs, verbose=1, validation_split=0.2,
    shuffle=False, validation_freq=1, batch_size=16, callbacks=[early_stopping])

Epoch 1/3000
81/81 [==============================] - 1s 3ms/step - loss: 0.2137 - accuracy: 0.6943 - mse: 0.2137 - val_loss: 0.1986 - val_accuracy: 0.6914 - val_mse: 0.1986
Epoch 2/3000
81/81 [==============================] - 0s 2ms/step - loss: 0.1938 - accuracy: 0.7128 - mse: 0.1938 - val_loss: 0.1928 - val_accuracy: 0.6914 - val_mse: 0.1928
Epoch 3/3000
81/81 [==============================] - 0s 2ms/step - loss: 0.1918 - accuracy: 0.7113 - mse: 0.1918 - val_loss: 0.1917 - val_accuracy: 0.6914 - val_mse: 0.1917
Epoch 4/3000
81/81 [==============================] - 0s 2ms/step - loss: 0.1911 - accuracy: 0.7098 - mse: 0.1911 - val_loss: 0.1912 - val_accuracy: 0.6914 - val_mse: 0.1912
Epoch 5/3000
81/81 [==============================] - 0s 2ms/step - loss: 0.1906 - accuracy: 0.7090 - mse: 0.1906 - val_loss: 0.1907 - val_accuracy: 0.6914 - val_mse: 0.1907
Epoch 6/3000
81/81 [==============================] - 0s 2ms/step - loss: 0.1902 - accuracy: 0.7098 - mse: 0.1902 - val_loss: 0.19

In [78]:
col_names = ['arquitectura', 'loss', 'val_loss', 'accuracy', 'val_accuracy', 'epoch']
#dff = pd.DataFrame([], columns=col_names)
dff = pd.read_csv(PATH + 'data/output/vals2.csv')
final_epoch=len(historico.history['loss'])-1
print(final_epoch)
tloss = historico.history['loss'][final_epoch-1]
print(tloss)
vloss = historico.history['val_loss'][final_epoch-1]
print(vloss)
tacc = historico.history['accuracy'][final_epoch-1]
print(tacc)
vacc = historico.history['val_accuracy'][final_epoch-1]
print(vacc)
datos = pd.DataFrame([[arch, tloss, vloss, tacc, vacc, final_epoch]], columns=col_names)
datos = pd.concat([dff, datos], ignore_index=True)
datos.to_csv(PATH + 'data/output/vals2.csv', index = False)

11
0.18882940709590912
0.18987923860549927
0.7097523212432861
0.6913580298423767


# Test

In [82]:
from keras.models import Sequential
from keras.layers import Dense
input_shape= (x_train.shape[1] ,)
final_model = Sequential()
arch = [20, 20, 0, "relu", 0.2]
final_model.add(Dense(arch[0], input_shape=input_shape, activation=arch[3]))
final_model.add(Dense(arch[1], input_shape=input_shape, activation=arch[3]))
final_model.add(Dense(2, activation='softmax'))
final_model.summary()

# CONFIGURAR MODELO Y ENTRENAMIENTO
from keras.callbacks import EarlyStopping

epochs = 20
final_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=arch[4], momentum=0), metrics=['accuracy','mse'], loss='mean_squared_error')
historico = final_model.fit(x_train, y_train_transformed, epochs=epochs, verbose=1, shuffle=False, batch_size=16)

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_84 (Dense)            (None, 20)                280       
                                                                 
 dense_85 (Dense)            (None, 20)                420       
                                                                 
 dense_86 (Dense)            (None, 2)                 42        
                                                                 
Total params: 742
Trainable params: 742
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
101/101 [==============================] - 0s 1ms/step - loss: 0.2122 - accuracy: 0.6906 - mse: 0.2122
Epoch 2/20
101/101 [==============================] - 0s 1ms/step - loss: 0.1946 - accuracy: 0.7073 - mse: 0.1946
Epoch 3/20
101/101 [==============================] - 0s 1ms/step - loss: 0.1931 - accuracy

In [83]:
# EVALUAR MODELO DEFINITIVO
train_results = final_model.evaluate(x_train, y_train_transformed, verbose=1)
test_results= final_model.evaluate(x_test, y_test_transformed, verbose=1)
#EL INDICE 0 ES EL LOSS. EL RESTO, LAS MÉTRICAS QUE SE HAN ESPECIFICADO AL COMPILAR EL MODELO.
# #EN ESTE CASO 'accuracy':1,'mse':2
print(f'Train results-Loss: {train_results[0]} -Accuracy: {train_results[1]} -MSE: {train_results[2]}')
print(f'Test results-Loss: {test_results[0]} -Accuracy: {test_results[1]} -MSE: {test_results[2]}')

25/25 [==============================] - 0s 1ms/step - loss: 0.1885 - accuracy: 0.6960 - mse: 0.1885
Train results-Loss: 0.18516811728477478 -Accuracy: 0.7035890817642212 -MSE: 0.18516811728477478
Test results-Loss: 0.1885344386100769 -Accuracy: 0.69597989320755 -MSE: 0.1885344386100769


In [84]:
# PREDICCIONES DE LAS CLASES
# PREDICCIONES EN BRUTO
raw_testPred= final_model.predict(x_test)
print(raw_testPred[:5])

# PREDICCIONES DE LA CLASE
testPred= np.argmax(raw_testPred, axis=1)
#TRANSFORMAR EL NÚMERO DE COLUMNA EN LA ETIQUETA
class_testPred= encoder.classes_[testPred]
print(class_testPred[:5])

25/25 [==============================] - 0s 1ms/step
[[0.36264595 0.6373541 ]
 [0.91705054 0.08294941]
 [0.38812006 0.61187994]
 [0.42710605 0.572894  ]
 [0.36394072 0.6360593 ]]
['human' 'bot' 'human' 'human' 'human']


In [87]:
test_targets = y_test
print(class_testPred[:6])
print(test_targets[:6])
comp_df = pd.DataFrame({"predicción": class_testPred, 'valor real': test_targets}, columns=['predicción','valor real'])
#guardar en fichero las salidas de test y target
comp_df.to_csv(PATH + 'data/output/salidas_test.csv', index=False)
#mostrar 10 primeras filas
print(comp_df.iloc[:10,:])

['human' 'bot' 'human' 'human' 'human' 'human']
['bot' 'human' 'human' 'human' 'human' 'bot']
  predicción valor real
0      human        bot
1        bot      human
2      human      human
3      human      human
4      human      human
5      human        bot
6      human      human
7      human        bot
8        bot      human
9      human        bot


In [88]:
# MATRIZ DE CONFUSIÓN
from sklearn.metrics import confusion_matrix, classification_report
cm=confusion_matrix(y_test,class_testPred)
print('Matriz de confusión')
print(cm)
print('Classification Report')
print(classification_report(y_test,class_testPred))

Matriz de confusión
[[ 20 444]
 [110 222]]
Classification Report
              precision    recall  f1-score   support

         bot       0.15      0.04      0.07       464
       human       0.33      0.67      0.44       332

    accuracy                           0.30       796
   macro avg       0.24      0.36      0.26       796
weighted avg       0.23      0.30      0.22       796

